In [1]:
import dotenv
from langchain import hub
from langchain.agents import AgentExecutor, Tool, create_openai_functions_agent
from langchain.prompts import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    PromptTemplate,
    SystemMessagePromptTemplate,
)
import time
import random
from langchain.schema.runnable import RunnablePassthrough
from langchain_community.vectorstores import Chroma
from langchain_core.output_parsers import StrOutputParser
from tools import get_current_wait_time
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain.document_loaders.csv_loader import CSVLoader



In [2]:
REVIEWS_CSV_PATH = "reviews.csv"
loader = CSVLoader(file_path = REVIEWS_CSV_PATH, source_column="review")
reviews = loader.load()

# Prompt

In [3]:
# dotenv.load_dotenv()
review_template_str = """Your job is to use patient
reviews to answer questions about their experience at
a hospital. Use the following context to answer questions.
Be as detailed as possible, but don't make up any information
that's not from the context. If you don't know an answer, say
you don't know.
{context}
"""


review_system_prompt = SystemMessagePromptTemplate(
    prompt=PromptTemplate(
        input_variables=["context"], template=review_template_str
    )
)

review_human_prompt = HumanMessagePromptTemplate(
    prompt=PromptTemplate(input_variables=["question"], template="{question}")
)
messages = [review_system_prompt, review_human_prompt]

review_prompt_template = ChatPromptTemplate(
    input_variables=["context", "question"], messages=messages
)

In [4]:
for i in review_prompt_template:
    print(i)

('name', None)
('input_variables', ['context', 'question'])
('optional_variables', [])
('input_types', {})
('output_parser', None)
('partial_variables', {})
('metadata', None)
('tags', None)
('messages', [SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], template="Your job is to use patient\nreviews to answer questions about their experience at\na hospital. Use the following context to answer questions.\nBe as detailed as possible, but don't make up any information\nthat's not from the context. If you don't know an answer, say\nyou don't know.\n{context}\n")), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['question'], template='{question}'))])
('validate_template', False)


# Model

In [5]:
from langchain_together import Together
from langchain_openai import ChatOpenAI

# chat_model = Together(
#     model="META-LLAMA/LLAMA-3-8B-CHAT-HF",
#     together_api_key="24cdbdf50106e08f6ba3328ac07f97a73eb440ae36da6cdd72f9b091ccca850a"
# )

chat_model = ChatOpenAI(
    base_url="https://api.together.xyz/v1",
    api_key="24cdbdf50106e08f6ba3328ac07f97a73eb440ae36da6cdd72f9b091ccca850a",
    # model="META-LLAMA/LLAMA-3-8B-CHAT-HF",
    model="mistralai/Mixtral-8x7B-Instruct-v0.1",
    # model="google/gemma-2-27b-it",
    
    # temperature=0
)
output_parser = StrOutputParser()

In [6]:
def get_current_wait_time(hospital: str) -> int | str:
    """Dummy function to generate fake wait times"""

    if hospital not in ["A", "B", "C", "D"]:
        return f"Hospital {hospital} does not exist"
    # Simulate API call delay
    time.sleep(1)

    return random.randint(0, 10000)

# Embeddings

In [7]:
from langchain_together.embeddings import TogetherEmbeddings
embeddings = TogetherEmbeddings(
    model="togethercomputer/m2-bert-80M-8k-retrieval",
    together_api_key="24cdbdf50106e08f6ba3328ac07f97a73eb440ae36da6cdd72f9b091ccca850a"                          
)

# Vector Store

In [8]:
# Load Chroma-vectordb
REVIEWS_CHROMA_PATH = "chroma_data (self-created)"
reviews_vector_db = Chroma(
    persist_directory=REVIEWS_CHROMA_PATH,
    embedding_function=embeddings,
)

In [9]:
reviews_vector_db
print(reviews_vector_db._collection.count())

prompt = hub.pull("rlm/rag-prompt")

1005


In [10]:
reviews_retriever = reviews_vector_db.as_retriever(k=10)

review_chain = (
    {"context": reviews_retriever, "question": RunnablePassthrough()}
    | review_prompt_template
    # | prompt
    | chat_model
    | StrOutputParser()
)

In [11]:
review_chain.invoke("what are you")

" I am a language model designed to understand and respond to questions or statements by generating human-like text. In this instance, I was initialized with a set of documents containing patient reviews of their experiences at various hospitals. I can use this context to answer questions about the reviews, such as details about the patient's experience with the hospital staff, food quality, cleanliness, and facilities. Please ask a specific question, and I will do my best to provide a detailed and accurate response, considering only the information present in the provided documents."

In [12]:
# review_chain.invoke("What do patients think about the triage system?")

In [13]:
docs = reviews_vector_db.max_marginal_relevance_search("what was the quality of food",k=3)
# docs

# Tools

In [34]:
tools = [
    Tool(
        name="Reviews",
        func=review_chain.invoke,
        description="""Useful when you need to answer questions
        about patient reviews or experiences at the hospital.
        Not useful for answering questions about specific visit
        details such as payer, billing, treatment, diagnosis,
        chief complaint, hospital, or physician information.
        Pass the entire question as input to the tool. For instance,
        if the question is "What do patients think about the triage system?",
        the input should be "What do patients think about the triage system?"
        """,
    ),
    Tool(
        name="Waits",
        func=get_current_wait_time,
        description="""Use when asked about current wait times
        at a specific hospital. This tool can only get the current
        wait time at a hospital and does not have any information about
        aggregate or historical wait times. This tool returns wait times in
        minutes. Do not pass the word "hospital" as input,
        only the hospital name itself. For instance, if the question is
        "What is the wait time at hospital A?", the input should be "A".
        """,
    ),
]

In [41]:
hospital_agent_prompt = hub.pull("hwchase17/openai-functions-agent")
# hospital_agent_prompt = hub.pull("hwchase17/openai-tools-agent")


agent_chat_model = ChatOpenAI(
    base_url="https://api.together.xyz/v1",
    api_key="24cdbdf50106e08f6ba3328ac07f97a73eb440ae36da6cdd72f9b091ccca850a",
    model="mistralai/Mixtral-8x7B-Instruct-v0.1",
    # model="META-LLAMA/LLAMA-3-8B-CHAT-HF",
    # model="google/gemma-2-27b-it",
    # temperature=0
)

from langchain.agents import create_tool_calling_agent
hospital_agent = create_tool_calling_agent(
    llm=agent_chat_model,
    prompt=hospital_agent_prompt,
    tools=tools,
)
# hospital_agent = create_openai_functions_agent(
#     llm=agent_chat_model,
#     prompt=hospital_agent_prompt,
#     tools=tools,
# )

hospital_agent_executor = AgentExecutor(
    agent=hospital_agent,
    tools=tools,
    return_intermediate_steps=True,
    verbose=True,
)

In [42]:
hospital_agent_executor.invoke({"input":"what are patient experiences at Pugh Rogers hospital?"})



> Entering new AgentExecutor chain...

Invoking: `Reviews` with `What are patient experiences at Pugh Rogers hospital?`


 Based on the provided context, there are two reviews about patients' experiences at Pugh-Rogers hospital.

The first patient, Jamie Jones, had a mixed experience at Pugh-Rogers hospital. They found the medical care to be good, but they were disappointed by the lack of communication between the hospital staff and themselves, which made their stay less enjoyable.

The second patient, Nicole Bradley, had a generally positive experience at Pugh-Rogers hospital. They appreciated the attentiveness of the medical team, but they found the noise from neighboring rooms to be disruptive to their recovery.

Therefore, while both patients received good medical care at Pugh-Rogers hospital, they had different issues with their overall experience, related to communication and noise levels, respectively. Based on the reviews provided, it seems that the hospital staff at Pugh-Rog

{'input': 'what are patient experiences at Pugh Rogers hospital?',
 'output': ' Based on the reviews provided, it seems that the hospital staff at Pugh-Rogers hospital are skilled at providing good medical care. However, there are areas for improvement in terms of communication and noise levels. These issues may have impacted the overall satisfaction of the patients during their stay. It would be beneficial for the hospital to address these concerns in order to enhance the overall patient experience.',
 'intermediate_steps': [(ToolAgentAction(tool='Reviews', tool_input='What are patient experiences at Pugh Rogers hospital?', log='\nInvoking: `Reviews` with `What are patient experiences at Pugh Rogers hospital?`\n\n\n', message_log=[AIMessageChunk(content='', additional_kwargs={'tool_calls': [{'index': 0, 'id': 'call_n29oxspnnr50szbwc32dwaim', 'function': {'arguments': '{\n      "__arg1": "What are patient experiences at Pugh Rogers hospital?"\n    }\n  ', 'name': 'Reviews'}, 'type': 'f

In [43]:
hospital_agent_executor.invoke({"input":"What is the wait time at hospital A?"})



> Entering new AgentExecutor chain...

Invoking: `Waits` with `A`


9122
Invoking: `Reviews` with `9122`


 The review with the visit ID 4397 is from Mark Solomon, who had a negative experience regarding parking availability at Pugh-Rogers hospital. According to Mark, the lack of parking spaces and the difficulty in finding one, particularly during peak hours, added unnecessary stress to his hospital visit.

Additionally, from the review with the visit ID 3263, Shannon Long, a patient at Walton LLC, was pleasantly surprised by the quality of food provided during her stay. The dietary staff even accommodated her special requests, which she considered a positive aspect of her hospital stay.

Heather Palmer, the patient from the review with the visit ID 6686 at Brown Inc, had a positive overall experience, but she was not satisfied with the food quality.

Lastly, Traci Carpenter, whose review has the visit ID 6305 at Pugh-Rogers, had a positive overall experience, with the medical staff

{'input': 'What is the wait time at hospital A?',
 'output': " That's really interesting! It's great to hear that Mark Solomon was pleased with the medical care he received at Pugh-Rogers hospital, but the lack of parking availability did impact his overall experience negatively. It's also good to note that Shannon Long from Walton LLC was happy with the dietary staff's accommodation of her special requests, which made her hospital stay more enjoyable.\n\nIt's important to consider factors like parking availability and food quality when evaluating the overall experience of a hospital visit, as they can significantly impact a patient's comfort and satisfaction. It's great that the medical staff at Pugh-Rogers and Brown Inc. were attentive to their patients' needs, but there is definitely room for improvement in other areas.\n\nThank you for sharing these reviews!",
 'intermediate_steps': [(ToolAgentAction(tool='Waits', tool_input='A', log='\nInvoking: `Waits` with `A`\n\n\n', message_lo

In [33]:
# # print(hospital_agent_executor)
# for i in hospital_agent_executor:
#     print(i)